<h1> Alexnet With Tensorflow </h1>

In [1]:
import tensorflow as tf

In [2]:
input = tf.placeholder(tf.float32, shape=(224,244))

layer1 = tf.nn.conv2d( [128,224,224,3] , [11,11,3,96], strides=[1,4,4,1], padding='valid')
layer1 = tf.nn.relu(layer1)

TypeError: Value passed to parameter 'input' has DataType int32 not in list of allowed values: float16, float32

In [11]:
device_type = '/gpu:0'

n_input = 224 * 224
n_output = 1000

with tf.device(device_type):
    weights = {
        'wc1' : tf.Variable(tf.truncated_normal([11,11,3,96], stddev=0.1)),
        'wc2' : tf.Variable(tf.truncated_normal([5,5,96, 256], stddev=0.1)),
        'wc3' : tf.Variable(tf.truncated_normal([3,3,256,384], stddev=0.1)),
        'wc4' : tf.Variable(tf.truncated_normal([3,3,384,384], stddev=0.1)),
        'wc5' : tf.Variable(tf.truncated_normal([3,3,384,256], stddev=0.1)),
        'wd1' : tf.Variable(tf.truncated_normal([224*224*256, 2048], stddev=0.1)),
        'wd2' : tf.Variable(tf.truncated_normal([2048,2048], stddev=0.1)),
        'wd3' : tf.Variable(tf.truncated_normal([2048,1000], stddev=0.1))
    }
    biases = {
        'bc1' : tf.Variable(tf.random_normal([96], stddev=0.1)),
        'bc2' : tf.Variable(tf.random_normal([256], stddev=0.1)),
        'bc3' : tf.Variable(tf.random_normal([384], stddev=0.1)),
        'bc4' : tf.Variable(tf.random_normal([384], stddev=0.1)),
        'bc5' : tf.Variable(tf.random_normal([256], stddev=0.1)),
        'bd1' : tf.Variable(tf.random_normal([2048], stddev=0.1)),
        'bd2' : tf.Variable(tf.random_normal([2048], stddev=0.1)),
        'bd3' : tf.Variable(tf.random_normal([1000], stddev=0.1))
    }
    
    def conv_basic(_input, _w, _b, _keepratio):
        #INPUT
        _input_r = tf.reshape(_input, shape=[-1,224,224,3])
        
        #CONV LAYER 1
        _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1,4,4,1], padding='valid')
        _conv1 = tf.nn.relu( tf.nn.bias_add(_conv1, _b['bc1']) )
        
        #CONV LAYER 2
        _conv2 = tf.nn.conv2d(_conv1, _w['wc2'], strides=[1,1,1,1], padding='valid')
        _conv2 = tf.nn.relu( tf.nn.bias_add(_conv2, _b['bc2']) )
        _pool2 = tf.nn.max_pool(_conv2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='valid')
        
        #CONV LAYER 3
        _conv3 = tf.nn.conv2d(_pool2,  _w['wc3'], strides = [1,1,1,1], padding='valid')
        _conv3 = tf.nn.relu( tf.nn.bias_add(_conv3, _b['bc3']))
        _pool3 = tf.nn.max_pool(_conv3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='valid')
        
        #CONV LAYER 4
        _conv4 = tf.nn.conv2d(_pool3, _w['wc4'], strides = [1,1,1,1], padding='valid')
        _conv4 = tf.nn.relu(tf.nn.bias_add(_conv4, _b['bc4']))
        
        #CONV LAYER 5
        _conv5 = tf.nn.conv2d(_conv4, _wc['wc5'], strides=[1,1,1,1], padding='valid')
        _conv5 = tf.nn.relu(tf.nn.bias_add(_conv5, _b['bc5']))
        _pool5 = tf.nn.max_pool(_conv5, ksize=[1,2,2,1], strides=[1,2,2,1], padding='valid')
        
        #VECTORIZE
        _dense1 = tf.reshape(_pool5, [-1, _w['wd1'].get_shape().as_list()[0]])
        
        #FULLY CONNECTED LAYER 1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        
        #FULLY CONNECTED LAYER 2
        _fc2 = tf.add( tf.matmul(_fc1, _w['wd2']), _b['bd2'])
        
        #FULLY CONNECTED LAYER 3
        _out = tf.add(tf.matmul(_fc2, _w['wd3']), _b['bd3'])
        
        #RETURN
        out = { 'input_r': _input_r, 'conv1': _conv1, 'conv2':_conv2, 'conv3':_conv3, 'conv4':_conv4, 'conv5':_conv5,
                   'dense1': _dense1, 'fc1':_fc1, 'fc2':_fc2, 'out':_out
              }
        return out

<h1> DEFINE COMPUTATIONAL GRAPH </h1>

In [6]:
#PLACEHOLDER


x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

#FUNCTIONS
with tf.device(device_type):
    _pred = conv_basic(x, weights, biases, keepratio)['out']
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
    optm = tf.train.MomentumOptimizer(0.0005,0.9).minimize(cost)
    _corr = tf.equal(tf.argmax(_pred,1) ,tf.argmax(y,1))
    accr = tf.reduce_mean(tf.cast(_corr,tf.float32))
    init = tf.global_variables_initializer()
    
#SAVER
save_step = 1
saver = tf.train.Saver(max_to_keep=3)
print("Graph READY")
    


NameError: name 'conv_basic' is not defined

<h1> OPTIMIZE </h1>

<h2> DO TRAIN OR NOT </h2>

In [7]:
do_train = 1
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(init)

NameError: name 'init' is not defined

In [8]:
training_epoch = 15
batch_size = 128
display_step = 1

if do_train == 1:
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int( /batch_size)
        
        #Loop over all batches
        for i in range(total_batch):
            batch_xs , batch_ys = 
            
            #Fit training using batch data
            sess.run(optm, feed_dict={x: batch_xs, y:batch_ys, keepratio:0.5})
            
            #Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y:batch_ys, keepratio: 1.0})/ total_batch
            
        #Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc = sess.run(accr, feed_dict={x: batch_xs, y:batch_ys, keepratio:1.})
            print("Training accuracy: %.3f" %(train_acc))
            test_acc = sess.run(accr, feed_dict={x: testimg, y:testlabel, keepratio:1.})
            print("Test accuracy: %.3f" %(test_acc))
            
        #Save Net
        if epoch % save_step == 0:
            saver.save(sess,"nets/alexNet_basic.ckpt-"+str(epoch))
            
    print("OPTIMIZATION FINISHED")

SyntaxError: invalid syntax (<ipython-input-8-b14a9f50630d>, line 8)

<h1> RESTORE </h1>

In [9]:
if do_train == 0:
    epoch = training_epochs-1
    saver.restore(sess, "nets/cnn_mnist_basic.ckpt-"+str(epoch))
    

<h1> COMPUTE TEST ACCURACY </h1>

In [10]:
test_acc = sess.run(accr, feed_dict={x: testimg, y:testlabel, keepratio:1.})
print("TEST ACCURACY: %.3f" %(test_acc))

NameError: name 'accr' is not defined